## Imports

In [385]:
import pandas as pd
import numpy as np
import re
import os
import folium

## Data Read-in

In [386]:
df = pd.read_csv('redfin_2023-04-04-10-04-08.csv',dtype='str')

## Data Clean

In [387]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [388]:
df = df.dropna(subset=['SOLD DATE'])

In [389]:
# Define list of desired months (excluding current month)
desired_months = ['February', 'March']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [390]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')

False    1812
Name: PRICE, dtype: int64
-------
False    1797
True       15
Name: $/SQUARE FEET, dtype: int64
-------


In [391]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [392]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [393]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
995,PAST SALE,March-28-2023,Condo/Co-op,2750 NE 183rd St #104,Aventura,FL,33160,255,2,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/2750-NE-183...,MARMLS,A11305707,N,Y,25.943908,-80.145145
800,PAST SALE,February-16-2023,Condo/Co-op,9441 SW 4th St #101,Miami,FL,33174,315,3,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/9441-SW-4th-St...,MARMLS,A11316416,N,Y,25.765477,-80.350522
1291,PAST SALE,February-15-2023,Condo/Co-op,1351 NE Miami Gardens Dr Unit 922E,Miami,FL,33179,15500,1,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1351-NE-Miami-...,MARMLS,A11297694,N,Y,25.945827,-80.174810
582,PAST SALE,March-3-2023,Condo/Co-op,13480 NE 6th Ave #310,North Miami,FL,33161,40000,1,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/13480-NE...,MARMLS,A11317232,N,Y,25.898940,-80.187168
906,PAST SALE,March-9-2023,Condo/Co-op,16425 Collins Ave #2,Sunny Isles Beach,FL,33160,40000,0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11313999,N,Y,25.927799,-80.121458
467,PAST SALE,March-29-2023,Condo/Co-op,20281 E Country Club Dr Unit CAB1,Aventura,FL,33180,50000,0,NaN,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/20281-E-Cou...,MARMLS,A11342462,N,Y,25.965506,-80.125855
1686,PAST SALE,February-10-2023,Condo/Co-op,13480 NE 6th Ave #205,North Miami,FL,33161,50000,2,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/13480-NE...,MARMLS,A11228625,N,Y,25.898940,-80.187168
531,PAST SALE,March-10-2023,Condo/Co-op,1401 NE Miami Gardens Dr #197,Miami,FL,33179,60000,0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1401-NE-Miami-...,MARMLS,A11305770,N,Y,25.945427,-80.172641
1811,PAST SALE,March-9-2023,Condo/Co-op,251 NE 187th St #827,Miami,FL,33179,65500,1,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/251-NE-187th-S...,MARMLS,A10680799,N,Y,25.946879,-80.196152
362,PAST SALE,February-24-2023,Condo/Co-op,19051 NE 2nd Ave #1505,Miami,FL,33179,89000,1,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/19051-NE-2nd-A...,MARMLS,A11327317,N,Y,25.949734,-80.197047


In [394]:
print(df_filtered['URL'].iloc[1291])

https://www.redfin.com/FL/Miami/1351-NE-Miami-Gardens-Dr-33179/unit-922E/home/43011337


In [395]:
# Correct the prices, if needed
df_filtered.at[995,'PRICE']=(255000)
df_filtered.at[800,'PRICE']=(315000)
df_filtered.at[1291,'PRICE']=(155000)

In [396]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [397]:
# Corrections, if needed
df_filtered.at[800,'$/SQUARE FEET']=(315000/1266)
df_filtered.at[995,'$/SQUARE FEET']=(255000/1073)

In [398]:
# # Find problem psf by searching for low values
# df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['ADDRESS','CITY','$/SQUARE FEET']]

In [399]:
print(df_filtered.URL.iloc[582])

https://www.redfin.com/FL/North-Miami/13480-NE-6th-Ave-33161/unit-310/home/43434030


In [400]:
# Corrections, if needed
#
#
#

In [401]:
# Drop sales that aren't condos but labeled as such
df_filtered = df_filtered.drop(1320)

## Make Maps

In [402]:
df_Current = df_filtered.loc[df_filtered['SOLD DATE'].str.contains('March')]

In [403]:
### Create a price column formatted as currency ###
df_Current['PRICE_AS_CURRENCY'] = df_Current['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_Current['YEAR BUILT'] = df_Current['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_Current['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_Current['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_9415/21604551.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Current['PRICE_AS_CURRENCY'] = df_Current['PRICE'].apply(lambda x: "${:,.0f}".format(x))
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_9415/21604551.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Current['YEAR BUILT'] = df_Current['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_9415/21604551.py:5: Set

In [404]:
df_Current = df_Current.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_Current['COLOR'] = ''
### Create RANK column ###
df_Current['RANK'] = 0
### Insert RANK values ###
df_Current['RANK'] = range(1, len(df_Current) + 1)
# use numpy to assign values to the 'COLOR' column
df_Current['COLOR'] = np.where(df_Current['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [405]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [406]:
### Drop the columns ###
df_Current = df_Current.drop(columns=columns_drop)

In [407]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [408]:
### Create map container ###
m = folium.Map(location=df_March[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10.5,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"March 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_Current.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [409]:
m.save('index.html')

## Summary Info

In [410]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [411]:
### Convert 'YEAR BUILT' back to integer ###
df_Current['YEAR BUILT'] = pd.to_numeric(df_Current['YEAR BUILT'])

In [412]:
df_Current.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'PRICE_SQUARE_FEET_AS_CURRENCY',
       'COLOR', 'RANK'],
      dtype='object')

In [413]:
df_Current['FULL_ADDRESS'] = df_Current['ADDRESS'] + ' ' + df_Current['CITY']

In [414]:
print(f"{ME}{BR}{df_Current.loc[df_Current['PRICE'].idxmax()]['LOCATION']}, {df_Current.loc[df_Current['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_Current.loc[df_Current['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_Current.loc[df_Current['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_Current.loc[df_Current['PRICE'].idxmax()]['YEAR BUILT']}")
print(f"{LE}{BR}{df_Current.loc[df_Current['PRICE'].idxmin()]['LOCATION']}, {df_Current.loc[df_Current['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_Current.loc[df_Current['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_Current.loc[df_Current['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_Current.loc[df_Current['PRICE'].idxmin()]['YEAR BUILT']}")

print(f"{MAX_PSF}{BR}{df_Current.loc[df_Current['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_Current.loc[df_Current['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_Current.loc[df_Current['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_Current.loc[df_Current['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_Current.loc[df_Current['$/SQUARE FEET'].idxmax()]['YEAR BUILT']}")
print(f"{MIN_PSF}{BR}{df_Current.loc[df_Current['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_Current.loc[df_Current['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_Current.loc[df_Current['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_Current.loc[df_Current['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_Current.loc[df_Current['$/SQUARE FEET'].idxmin()]['YEAR BUILT']}")

print(f"{Newest}{BR}{df_Current.loc[df_Current['YEAR BUILT'].idxmax()]['LOCATION']}, {df_Current.loc[df_Current['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_Current.loc[df_Current['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_Current.loc[df_Current['YEAR BUILT'].idxmax()]['$/SQUARE FEET']} psf | Year built: {df_Current.loc[df_Current['YEAR BUILT'].idxmax()]['YEAR BUILT']}")
print(f"{Oldest}{BR}{df_Current.loc[df_Current['YEAR BUILT'].idxmin()]['LOCATION']}, {df_Current.loc[df_Current['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_Current.loc[df_Current['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_Current.loc[df_Current['YEAR BUILT'].idxmin()]['$/SQUARE FEET']} psf | Year built: {df_Current.loc[df_Current['YEAR BUILT'].idxmin()]['YEAR BUILT']}")

Most Expensive
OCEANSIDE @ FISHER ISL CO, 7954 Fisher Island Dr #7954 Miami Beach | Price $21,000,000 | $3,078 psf | Year built: 1989
Least Expensive
OCEANIA I CONDO, 16425 Collins Ave #2 Sunny Isles Beach | Price $40,000 | $250 psf | Year built: 1990
Highest Price Per Square Foot
CONTINUUM ON SOUTH BEACH, 50 S Pointe Dr #2802 Miami Beach | Price $10,150,000 | $4,055 psf | Year built: 2008
Lowest Price Per Square Foot
ZURICH CONDO, 13480 NE 6th Ave #310 North Miami | Price $40,000 | $55 psf | Year built: 1974
Newest
Estates at Acqualina, 17975 Collins Ave #1202 Sunny Isles Beach | Price $8,950,000 | $nan psf | Year built: 2023
Oldest
PALM GARDEN CONDO, 760 Meridian Ave #4 Miami Beach | Price $240,000 | $471.0 psf | Year built: 1923


## Time on Market Calculator

In [415]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [416]:
from datetime import datetime, timedelta

date1 = datetime(2022, 12, 6) ## List (Earlier) date
date2 = datetime(2023, 3, 24) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

108


## Map URL Snagger

In [417]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [418]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_month_ending_march_2023


## Get Summary Data

In [419]:
# print('SALES INFO')
# print(f'Number of sales: {len(df_Current)}')
# print('--------')
# print(f'Total sale price: ${df_Current["PRICE"].sum():,.0f}')
# print('--------')
# print(f'Median sale price: ${df_Current["PRICE"].median():,.0f}')
# print('--------')
# print(f'Max sale price: ${df_Current["PRICE"].max():,.0f}')
# print('--------')
# print(f'Min sale price: ${df_Current["PRICE"].min():,.0f}')
# print('------------------------------------------------')
# print('PSF INFO')
# print(f'Max price per square foot: ${df_Current["$/SQUARE FEET"].max():,.0f}')
# print('--------')
# print(f'Min price per square foot: ${df_Current["$/SQUARE FEET"].min():,.0f}')
# print('--------')
# print(f'Median price per square foot: ${df_Current["$/SQUARE FEET"].median():,.0f}')
# print('------------------------------------------------')
# print('CONDO AGES')
# print(f'Newest building: {df_Current["YEAR BUILT"].max()}')
# print('----------')
# print(f'Oldest building: {df_Current["YEAR BUILT"].min()}')
# print('----------')
# print(f'Average building age: {df_Current["YEAR BUILT"].mean()}')

In [420]:
df_filtered

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,PAST SALE,February-24-2023,Condo/Co-op,1627 Brickell Ave #2802,Miami,FL,33129,690000,2,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1627-Brickell-...,MARMLS,A11295806,N,Y,25.755987,-80.194088
1,PAST SALE,March-28-2023,Condo/Co-op,520 Brickell Key Dr Unit A1716,Miami,FL,33131,1200000,3,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/520-Brickell-K...,MARMLS,A11296884,N,Y,25.767170,-80.184935
2,PAST SALE,February-15-2023,Condo/Co-op,7600 Collins Ave #1205,Miami Beach,FL,33141,450000,1,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7600-Col...,MARMLS,A11286310,N,Y,25.861730,-80.121451
3,PAST SALE,February-23-2023,Condo/Co-op,251 172nd St #109,Sunny Isles Beach,FL,33160,240000,1,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/25...,MARMLS,A11347631,N,Y,25.935538,-80.124742
4,PAST SALE,March-24-2023,Condo/Co-op,22551 SW 88th Pl Unit 205-1,Cutler Bay,FL,33190,260000,2,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Cutler-Bay/22551-SW-...,MARMLS,A11359966,N,Y,25.558184,-80.334355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,PAST SALE,March-22-2023,Condo/Co-op,2301 Collins Ave #609,Miami Beach,FL,33139,2690000,2,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2301-Col...,MARMLS,A11021370,N,Y,25.798716,-80.126897
1808,PAST SALE,February-27-2023,Condo/Co-op,312 Ocean Dr Unit BH5,Miami Beach,FL,33139,3200000,3,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/312-Ocea...,MARMLS,A10813838,N,Y,25.772352,-80.132767
1809,PAST SALE,February-24-2023,Condo/Co-op,6515 Collins Ave #808,Miami Beach,FL,33141,880000,2,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6515-Col...,MARMLS,A10914567,N,Y,25.848322,-80.119687
1810,PAST SALE,March-22-2023,Condo/Co-op,18975 Collins Ave #2902,Sunny Isles Beach,FL,33160,4200000,4,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A10908812,N,Y,25.952128,-80.119708


In [421]:
# Extract month from SOLD DATE column
df_filtered['SOLD MONTH'] = df_filtered['SOLD DATE'].str.split('-').str[0]

In [422]:
df_filtered['SOLD MONTH'].value_counts()

March       1034
February     777
Name: SOLD MONTH, dtype: int64